In [108]:
import pandas as pd

In [109]:
import sklearn

In [110]:
import sklearn_crfsuite

In [111]:
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

In [112]:
df1 = pd.read_json('scenarios-labeled.json', encoding = "ISO-8859-1")

In [113]:
df = pd.read_csv('scenarios-labeled.csv', encoding = "ISO-8859-1")
df.head(10)

,Sentence,Token,POS,Tag


In [114]:
import json
file = open("scenarios-labeled.json", 'r')
data = json.load(file)
x = list(data.values())[0]['words']
#x[0]

In [115]:
import sys
!{sys.executable} -m pip install nltk
import nltk
nltk.download('universal_tagset')
nltk.download('averaged_perceptron_tagger')

index = 0
for x in list(data.values()):
    word_list = x['words']
    y = nltk.pos_tag(word_list, tagset='universal')
    for word in word_list:
        df.at[index, 'Token'] = word
        i = 0
        df.at[index, 'POS'] = y[i][1]
        i += 1
        index += 1


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/vaishnavi/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/vaishnavi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [116]:
df.Token

0          From
1          this
2        screen
3             ,
4             I
          ...  
56026    social
56027      life
56028       out
56029     there
56030         .
Name: Token, Length: 56031, dtype: object

In [117]:
df.POS

0        ADP
1        ADP
2        ADP
3        ADP
4        ADP
        ... 
56026    ADJ
56027    ADJ
56028    ADJ
56029    ADJ
56030    ADJ
Name: POS, Length: 56031, dtype: object

In [118]:
index = 0
for x in list(data.values()):
    codes_list = x['codes']
    for code in codes_list:
        df.at[index, 'Tag'] = code
        index += 1

df.Tag

0        O
1        O
2        O
3        O
4        O
        ..
56026    O
56027    O
56028    O
56029    O
56030    O
Name: Tag, Length: 56031, dtype: object

In [119]:
sentence_number = 1
flag = 0
for i in range(0, 56031):
    if(df.at[i, 'Token'] == "." or df.at[i, 'Token'] == "?" or df.at[i, 'Token'] == "!"):
        flag = 1
        df.at[i, 'Sentence'] = sentence_number
    if(flag == 1):
        sentence_number += 1
        flag = 0
    else:
        df.at[i, 'Sentence'] = sentence_number

df.Sentence

0           1
1           1
2           1
3           1
4           1
         ... 
56026    2420
56027    2420
56028    2420
56029    2420
56030    2420
Name: Sentence, Length: 56031, dtype: object

In [120]:
df.head(10)

,Sentence,Token,POS,Tag
0,1,From,ADP,O
1,1,this,ADP,O
2,1,screen,ADP,O
3,1,",",ADP,O
4,1,I,ADP,O
5,1,like,ADP,O
6,1,to,ADP,O
7,1,search,ADP,O
8,1,for,ADP,O
9,1,anything,ADP,O


In [121]:
df1.head(10)

,MAS-G-0001,MAS-G-0002,MAS-G-0003,MAS-G-0004,MAS-G-0005,MAS-G-0006,MAS-G-0007,MAS-G-0008,MAS-G-0009,MAS-G-0010,...,MAS-P-0291,MAS-P-0292,MAS-P-0293,MAS-P-0294,MAS-P-0295,MAS-P-0296,MAS-P-0297,MAS-P-0298,MAS-P-0299,MAS-P-0300
id,MAS-G-0001,MAS-G-0002,MAS-G-0003,MAS-G-0004,MAS-G-0005,MAS-G-0006,MAS-G-0007,MAS-G-0008,MAS-G-0009,MAS-G-0010,...,MAS-P-0291,MAS-P-0292,MAS-P-0293,MAS-P-0294,MAS-P-0295,MAS-P-0296,MAS-P-0297,MAS-P-0298,MAS-P-0299,MAS-P-0300
clean_text,"From this screen, I like to search for anythin...",I always keep a track of which games are going...,On this screen I want to put together a shoppi...,I use this screen by reading the posts to see ...,This is the general screen on wild rift that I...,I use this screen to add items to an errands l...,This screen allows me to get the currency conv...,This page/screen helps me choose which videos ...,The goal that I wish to achieve on the Popular...,"To find out what is going on with the world, I...",...,I would use this screen to remove my personal ...,I use this page as my ledger when I am seeing ...,This is the profile section of Instagram where...,Sunday Evenings I usually look at my daily ste...,I use this screen to edit the permissions i gi...,The screen shot that is pictured above is the ...,I use this screen to edit my screen name and g...,I can either change my phone number or delete ...,The goal of using this screen is to order some...,"Most often, I use this screen whenever I am th..."
words,"[From, this, screen, ,, I, like, to, search, f...","[I, always, keep, a, track, of, which, games, ...","[On, this, screen, I, want, to, put, together,...","[I, use, this, screen, by, reading, the, posts...","[This, is, the, general, screen, on, wild, rif...","[I, use, this, screen, to, add, items, to, an,...","[This, screen, allows, me, to, get, the, curre...","[This, page, /, screen, helps, me, choose, whi...","[The, goal, that, I, wish, to, achieve, on, th...","[To, find, out, what, is, going, on, with, the...",...,"[I, would, use, this, screen, to, remove, my, ...","[I, use, this, page, as, my, ledger, when, I, ...","[This, is, the, profile, section, of, Instagra...","[Sunday, Evenings, I, usually, look, at, my, d...","[I, use, this, screen, to, edit, the, permissi...","[The, screen, shot, that, is, pictured, above,...","[I, use, this, screen, to, edit, my, screen, n...","[I, can, either, change, my, phone, number, or...","[The, goal, of, using, this, screen, is, to, o...","[Most, often, ,, I, use, this, screen, wheneve..."
codes,"[O, O, O, O, O, O, O, O, O, O, O, B-Noun Phras...","[O, O, O, O, O, O, B-Questions, I-Questions, I...","[O, O, O, O, O, O, O, O, O, B-Noun Phrase, I-N...","[O, O, O, O, O, O, O, B-Noun Phrase, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, B-Noun Phrase, O, O, B-Noun...","[O, O, O, O, O, O, O, B-Noun Phrase, I-Noun Ph...","[O, O, O, O, O, O, O, O, B-Noun Phrase, O, O, ...","[O, O, O, O, O, O, O, O, O, O, B-Noun Phrase, ...","[O, O, O, B-Questions, I-Questions, I-Question...",...,"[O, O, O, O, O, O, O, O, B-Noun Phrase, I-Noun...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-Noun...","[O, O, O, B-Noun Phrase, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, B-Noun Phrase, I-Noun...","[O, O, O, O, O, O, O, B-Noun Phrase, O, O, O, ...","[O, O, O, O, O, O, O, O, O, B-Noun Phrase, O, ...","[O, O, O, O, O, O, O, B-Noun Phrase, I-Noun Ph...","[O, O, O, O, O, B-Noun Phrase, I-Noun Phrase, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-N..."


In [122]:
print(df)

      Sentence   Token  POS Tag
0            1    From  ADP   O
1            1    this  ADP   O
2            1  screen  ADP   O
3            1       ,  ADP   O
4            1       I  ADP   O
...        ...     ...  ...  ..
56026     2420  social  ADJ   O
56027     2420    life  ADJ   O
56028     2420     out  ADJ   O
56029     2420   there  ADJ   O
56030     2420       .  ADJ   O

[56031 rows x 4 columns]


In [123]:
df.describe()

,Sentence,Token,POS,Tag
count,56031,56031,56031,56031
unique,2420,4112,10,8
top,814,I,PRON,O
freq,206,2646,24152,47097


In [124]:
#Displaying the unique Tags
df['Tag'].unique()

array(['O', 'B-Noun Phrase', 'I-Noun Phrase', 'B-Questions',
       'I-Questions', 'B-Complex Terms', 'I-Complex Terms', '-'],
      dtype=object)

In [125]:
#Checking null values, if any.
df.isnull().sum()

Sentence    0
Token       0
POS         0
Tag         0
dtype: int64

In [126]:
df = df.fillna(method = 'ffill')

In [127]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Token'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [128]:
#Displaying one full sentence
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'From this screen , I like to search for anything from recipes , to home decor , to people , etc .'

In [129]:
#sentence with its pos and tag.
sent = getter.get_text()
print(sent)

None


In [130]:
sentences = getter.sentences

In [131]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [132]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [133]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [134]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)

In [135]:
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass

In [136]:
y_pred = crf.predict(X_test)

In [137]:
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

0.8962955798933244


In [138]:
from sklearn_crfsuite.metrics import flat_recall_score
recall_score = flat_recall_score(y_test, y_pred, average = 'weighted')
print(recall_score)

0.9048751486325802


In [139]:
from sklearn_crfsuite.metrics import flat_precision_score
precision_score = flat_precision_score(y_test, y_pred, average = 'weighted')
print(precision_score)

0.8932764662989274


/usr/local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
